In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('/kaggle/input/ghouls-goblins-and-ghosts-boo/train.csv.zip')
train_df.head()

In [ ]:
train_df.shape

In [ ]:
train_df.info()

In [ ]:
train_df['type'].value_counts()

In [ ]:
test_df = pd.read_csv('/kaggle/input/ghouls-goblins-and-ghosts-boo/test.csv.zip')
test_df.head()

In [ ]:
test_df.shape

In [ ]:
test_df.info()

In [ ]:
train_df['bone_length'].plot(kind='hist')

In [ ]:
test_df['bone_length'].plot(kind='hist')

In [ ]:
train_df['rotting_flesh'].plot(kind='hist')

In [ ]:
test_df['rotting_flesh'].plot(kind='hist')

In [ ]:
train_df['hair_length'].plot(kind='hist')

In [ ]:
test_df['hair_length'].plot(kind='hist')

In [ ]:
train_df['has_soul'].plot(kind='hist')

In [ ]:
test_df['has_soul'].plot(kind='hist')

In [ ]:
train_df['color'].value_counts()

In [ ]:
test_df['color'].value_counts()

In [ ]:
train_df = pd.concat([train_df,pd.get_dummies(train_df['color'])],axis = 1)
train_df.drop('color', axis=1, inplace = True)
train_df.head()

In [ ]:
test_df = pd.concat([test_df,pd.get_dummies(test_df['color'])],axis = 1)
test_df.drop('color', axis=1, inplace = True)
test_df.head()

In [ ]:
X = train_df.drop(['id','type'], axis = 1)
y = pd.get_dummies(train_df['type'])

In [ ]:
y.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
from tensorflow import keras

In [ ]:
from keras.layers import Dense, Activation
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Dense(80, input_shape=(X.shape[1],)))
model.add(Activation('relu'))
model.add(Dense(50))
model.add(Activation('relu'))
model.add(Dense(50))
model.add(Activation('relu'))
model.add(Dense(50))
model.add(Activation('relu'))
model.add(Dense(20))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('relu'))
model.add(Dense(3))
model.add(Activation('softmax'))
model.summary()


In [ ]:
model.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
r = model.fit(X_train,y_train,
             batch_size=16,
             epochs=50,
             verbose=2,
             validation_data=(X_test,y_test))

In [ ]:
plt.plot(r.history['accuracy'],'r',label='Training accuracy')
plt.plot(r.history['val_accuracy'],'y',label='Validation accuracy')
plt.legend()

In [ ]:
prediction = model.predict(test_df.drop('id',axis=1))

In [ ]:
final_pred = [np.argmax(i) for i in prediction]

In [ ]:
result = pd.DataFrame({'id':test_df['id'], 'type': final_pred})
result.head()

In [ ]:
result['type'].replace(to_replace=[0,1,2], value = ['Ghost','Ghoul','Goblin'],inplace=True)
result.head()

In [ ]:
result.to_csv('../working/submission.csv', index = False)